In [108]:
import pandas as pd
from datasets import Dataset 
import re
from io import StringIO
import csv

In [109]:
# load data into datafram 

title_label = pd.read_csv('title_labels.txt', quotechar='"', quoting=csv.QUOTE_NONNUMERIC, on_bad_lines='skip', names=['Title', 'Label'])

title_label['Label'] = pd.to_numeric(title_label['Label'], errors='coerce')

title_label_cleaned = title_label.dropna(subset=['Label'], axis=0, how='any')

title_label_cleaned['Label'] = title_label_cleaned['Label'].astype(int)

print(title_label_cleaned.dtypes)

title_label_dataset = Dataset.from_pandas(title_label_cleaned)

print(title_label_dataset)

# Convert label column to integer
#$title_label_dataset = title_label_dataset.map(lambda x: {'Label': int(x['Label'])})


Title    object
Label     int64
dtype: object
Dataset({
    features: ['Title', 'Label', '__index_level_0__'],
    num_rows: 529490
})


/var/folders/p1/nyppnjq941xgy5c34y__sdkc0000gn/T/ipykernel_31843/3020978255.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  title_label = pd.read_csv('title_labels.txt', quotechar='"', quoting=csv.QUOTE_NONNUMERIC, on_bad_lines='skip', names=['Title', 'Label'])
/var/folders/p1/nyppnjq941xgy5c34y__sdkc0000gn/T/ipykernel_31843/3020978255.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title_label_cleaned['Label'] = title_label_cleaned['Label'].astype(int)


In [110]:
print(title_label_dataset[:10])

{'Title': ['Stocks That Hit 52-Week Highs On Friday', 'Stocks That Hit 52-Week Highs On Wednesday', '71 Biggest Movers From Friday', "46 Stocks Moving In Friday's Mid-Day Session", 'B of A Securities Maintains Neutral on Agilent Technologies, Raises Price Target to $88', 'CFRA Maintains Hold on Agilent Technologies, Lowers Price Target to $85', 'UBS Maintains Neutral on Agilent Technologies, Raises Price Target to $87', 'Agilent Technologies shares are trading higher after the company reported better-than-expected Q2 EPS and sales results.', 'Wells Fargo Maintains Overweight on Agilent Technologies, Raises Price Target to $95', '10 Biggest Price Target Changes For Friday'], 'Label': [2, 0, 2, 2, 2, 2, 2, 2, 2, 2], '__index_level_0__': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}


In [111]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels = 3)

In [123]:
def tokenize_function(examples):
    return tokenizer(examples['Title'], padding = 'max_length', truncation=True, max_length=512)

tokenized_dataset = title_label_dataset.map(tokenize_function, batched=True)
print(tokenized_dataset)


Map:   0%|          | 0/529490 [00:00<?, ? examples/s]

Dataset({
    features: ['Title', 'Label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 529490
})


In [129]:
#tokenized_dataset = tokenized_dataset.rename_column('Label', 'labels')
tokenized_dataset = tokenized_dataset.remove_columns([col for col in title_label_dataset.column_names if col.startswith('__')])

print(tokenized_dataset)
training_args = TrainingArguments(output_dir='./results', evaluation_strategy='epoch', learning_rate=2e-5, 
                                  per_device_train_batch_size=8, per_device_eval_batch_size=8, 
                                  num_train_epochs=3, weight_decay=0.01, remove_unused_columns=False)


Dataset({
    features: ['Title', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 529490
})


/Users/losangeles/miniconda3/lib/python3.11/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [130]:
train_set = tokenized_dataset.select(range(475000))
eval_set = tokenized_dataset.select(range(475000, len(title_label_dataset)))
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_set,
                  eval_dataset=eval_set)

trainer.train()

  0%|          | 0/178125 [00:00<?, ?it/s]

{'loss': 0.6227, 'grad_norm': 9.972365379333496, 'learning_rate': 1.9943859649122806e-05, 'epoch': 0.01}
{'loss': 0.5095, 'grad_norm': 0.8242043256759644, 'learning_rate': 1.9887719298245615e-05, 'epoch': 0.02}
{'loss': 0.4792, 'grad_norm': 0.29578152298927307, 'learning_rate': 1.9831578947368423e-05, 'epoch': 0.03}
{'loss': 0.4863, 'grad_norm': 5.147058486938477, 'learning_rate': 1.977543859649123e-05, 'epoch': 0.03}
{'loss': 0.4617, 'grad_norm': 1.9013699293136597, 'learning_rate': 1.9719298245614036e-05, 'epoch': 0.04}
{'loss': 0.454, 'grad_norm': 12.737407684326172, 'learning_rate': 1.9663157894736844e-05, 'epoch': 0.05}
{'loss': 0.436, 'grad_norm': 0.4781021475791931, 'learning_rate': 1.960701754385965e-05, 'epoch': 0.06}
{'loss': 0.4371, 'grad_norm': 7.440695285797119, 'learning_rate': 1.9550877192982457e-05, 'epoch': 0.07}
{'loss': 0.4374, 'grad_norm': 5.545661926269531, 'learning_rate': 1.9494736842105265e-05, 'epoch': 0.08}
{'loss': 0.4305, 'grad_norm': 8.059834480285645, 'lea

SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [ ]:
model.save_pretrained('./fin_bert_3_labels')
tokenizer.save_pretrained('./fin_bert_3_labels')


In [ ]:
predictions = trainer.predict(tokenized_dataset)
print(predictions)